In [1]:
import pandas as pd
import numpy as np
df=pd.read_excel("Data_For Model 20230107.xlsx")

In [2]:
df["Label"].value_counts()

Label
IS    8131
ES    3093
iS       1
b        1
Name: count, dtype: int64

In [3]:
tmpd={"IS":0,"ES":1}

In [4]:
df["label"]=df["Label"].map(tmpd)

In [5]:
df=df.dropna(subset=["label"])

In [6]:
df["label"].value_counts()

label
0.0    8131
1.0    3093
Name: count, dtype: int64

In [7]:
df["text"]=df["Text"]

In [8]:
df

,Unnamed: 0,Text,Label,label,text
0,NaN,别担心，这种情况是可以解决的,ES,1.0,别担心，这种情况是可以解决的
1,NaN,遇到这种事情是很麻烦的，不是三言两语可以解决的，要具体问题具体对待，希望你们和睦相处，开心快乐！,ES,1.0,遇到这种事情是很麻烦的，不是三言两语可以解决的，要具体问题具体对待，希望你们和睦相处，开心快乐！
2,NaN,我的已经到了，希望大家的也马上到达。,ES,1.0,我的已经到了，希望大家的也马上到达。
3,NaN,呵呵，人力搜索中——查找！,IS,0.0,呵呵，人力搜索中——查找！
4,NaN,寻找共同爱好，比如游戏等,IS,0.0,寻找共同爱好，比如游戏等
...,...,...,...,...,...
11221,NaN,这个也很好玩耶！,ES,1.0,这个也很好玩耶！
11222,NaN,太强了,ES,1.0,太强了
11223,NaN,我非常赞同这位同学的看法,ES,1.0,我非常赞同这位同学的看法
11224,NaN,我觉得您的想法和思维导图做的不错！,ES,1.0,我觉得您的想法和思维导图做的不错！


#### 去除停用词

In [9]:
import re
import jieba_fast as jieba
ting=[i.strip() for i in open("stop_words_ch-停用词表.txt",encoding='gbk').readlines()]
def fenci(text):
    return [i for i in jieba.cut(str(text)) if i not in ting] 

df["text"]=df["text"].apply(lambda x:fenci(x))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\jhg\AppData\Local\Temp\jieba.cache
Loading model cost 1.086 seconds.
Prefix dict has been built succesfully.


In [10]:
df["text"]=df["text"].apply(lambda x:" ".join(x))

### tfidf

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()  
vectors=vectorizer.fit_transform(df["text"])#训练数据集并且得到tfidf分数

D:\anaconda\root\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [12]:
from sklearn.feature_selection import SelectKBest,f_classif 
selector=SelectKBest(score_func=f_classif,k=5000)#假设性检验  得到和标签最相关的2000个类别
selector.fit(vectors,df["label"])
traindata=selector.transform(vectors).todense()

### 交叉验证

In [13]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
kf = KFold(n_splits=5, shuffle=True, random_state=42)
rm=RandomForestClassifier()
X=traindata
y=df["label"].values
d={"IS":0,"ES":1}
reports=[]
true_labels_all = []
predicted_labels_all = []

### 每一折的交叉验证报告 手动平均即为交叉验证的平均结果

In [21]:
for train_index, test_index in kf.split(X,y):

    # 划分训练集和测试集
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # 训练模型
    rm.fit(X_train, y_train)
    
    # 进行预测
    y_pred_test = rm.predict(X_test)
    
    # 生成分类报告
    report = classification_report(y_test, y_pred_test,digits=4,target_names=list(d.keys()))
    reports.append(report)
    print(f"Classification report for fold:\n{report}\n")
    true_labels_all.extend(y_test)
    predicted_labels_all.extend(y_pred_test)

D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Classification report for fold:
              precision    recall  f1-score   support

          IS     0.9063    0.9283    0.9172      1646
          ES     0.7889    0.7362    0.7617       599

    accuracy                         0.8771      2245
   macro avg     0.8476    0.8323    0.8394      2245
weighted avg     0.8750    0.8771    0.8757      2245




D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Classification report for fold:
              precision    recall  f1-score   support

          IS     0.9019    0.9170    0.9094      1614
          ES     0.7781    0.7448    0.7611       631

    accuracy                         0.8686      2245
   macro avg     0.8400    0.8309    0.8353      2245
weighted avg     0.8671    0.8686    0.8677      2245




D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Classification report for fold:
              precision    recall  f1-score   support

          IS     0.9104    0.9298    0.9200      1639
          ES     0.7986    0.7525    0.7749       606

    accuracy                         0.8820      2245
   macro avg     0.8545    0.8412    0.8474      2245
weighted avg     0.8802    0.8820    0.8808      2245




D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Classification report for fold:
              precision    recall  f1-score   support

          IS     0.8975    0.9289    0.9129      1631
          ES     0.7917    0.7182    0.7532       614

    accuracy                         0.8713      2245
   macro avg     0.8446    0.8236    0.8331      2245
weighted avg     0.8686    0.8713    0.8692      2245




D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Classification report for fold:
              precision    recall  f1-score   support

          IS     0.8946    0.9169    0.9056      1601
          ES     0.7794    0.7309    0.7544       643

    accuracy                         0.8636      2244
   macro avg     0.8370    0.8239    0.8300      2244
weighted avg     0.8616    0.8636    0.8623      2244


              precision    recall  f1-score   support

          IS     0.9063    0.9283    0.9172      1646
          ES     0.7889    0.7362    0.7617       599

    accuracy                         0.8771      2245
   macro avg     0.8476    0.8323    0.8394      2245
weighted avg     0.8750    0.8771    0.8757      2245

              precision    recall  f1-score   support

          IS     0.9019    0.9170    0.9094      1614
          ES     0.7781    0.7448    0.7611       631

    accuracy                         0.8686      2245
   macro avg     0.8400    0.8309    0.8353      2245
weighted avg     0.8671    0.8686    0.86

In [ ]:
true_labels_all = np.array(true_labels_all)
predicted_labels_all = np.array(predicted_labels_all)

# 创建一个DataFrame
results_df = pd.DataFrame({
    'True_Label': true_labels_all,
    'Predicted_Label': predicted_labels_all
})

# 将DataFrame保存为CSV文件
results_df.to_csv('rm_cross_validation.csv', index=False)

### 获得交叉验证的平均结果(更常用)

In [23]:
y_pred = cross_val_predict(rm, X, y, cv=kf)
# 生成总体的分类报告
print(classification_report(y, y_pred,digits=4,target_names=list(d.keys())))

D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
D:\anaconda\root\lib\site-packages\sklearn\utils\validatio

              precision    recall  f1-score   support

         0.0       0.90      0.92      0.91      8131
         1.0       0.79      0.74      0.76      3093

    accuracy                           0.87     11224
   macro avg       0.84      0.83      0.84     11224
weighted avg       0.87      0.87      0.87     11224

